In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, get_trajs, speed_windowed, theta_windowed, get_smooth_trajs, get_velocities
                   
show_verb = True
save_verb = True
anim_show_verb = False

In [2]:
rawTrajs = pd.read_parquet("../tracking/results/parquet/pre_merge_tracking.parquet")
red_particle_idx = 17
rawTrajs.loc[rawTrajs.particle != red_particle_idx, ["color"]] = "#00007F"
rawTrajs.loc[rawTrajs.particle == red_particle_idx, ["color"]] = "#FF0000"
colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nDrops = len(rawTrajs.loc[rawTrajs.frame==0])
nFrames = max(rawTrajs.frame) + 1
print(f"nDrops:{nDrops}")
print(f"nFrames:{nFrames} --> {nFrames/10:.2f} s")


# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print(f"window of {window/10} s, stride of {stride/10} s")
startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"number of steps: {nSteps}")


# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

nDrops:50
nFrames:32000 --> 3200.00 s
window of 320.0 s, stride of 10.0 s
number of steps: 288


In [3]:
# Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

# MSD

In [4]:
show_verb = False
run_windowed_analysis = False
animated_plot_verb = False
print(f"MSD Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")

%run ./analysis_modules/msd.ipynb

MSD Analysis: show_verb = False, run_windowed_analysis = False, animated_plot_verb = False
Global IMSD
Global EMSD
Raw trajs - Blue Particles: 1.179 ± 0.002, Red Particle: 0.762 ± 0.002
Smooth trajs - Blue Particles: 1.183 ± 0.002, Red Particle: 0.765 ± 0.002
Windowed IMSD
Windowed EMSD


# SPEED & TURNING ANGLES DISTRIBUTIONS

In [5]:
show_verb = False
animated_plot_verb = False
print(f"Speed and Turning Angles Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/speed_and_turning_angles.ipynb

Speed and Turning Angles Analysis: show_verb = False, animated_plot_verb = False

 Global speed distribution analysis
Raw Trajectories
Blue Particles σ: 3.0812678386034857 ± 0.015939935826104385
Red Particle σ: 7.3063902778733985 ± 0.1325638679530156
Smooth Trajectories
Blue Particles σ: 2.6247331712013717 ± 0.016982879149678404
Red Particle σ: 7.501490328804058 ± 0.14385893343130896

 Windowed speed distribution Analysis


100%|██████████| 288/288 [01:06<00:00,  4.33it/s]



 Global turning angles analysis
Raw Trajectories
Blue Particles σ: 2.917682835740027 ± 0.30949859447841155, μ: -0.0007054159678248849 ± 0.4540618048966584
Red Particle σ: 2.4384780096960923 ± 0.15247337370262667, μ: 0.017968089879776165 ± 0.19547887167646438
Smooth Trajectories
Blue Particles σ: 0.4532020875829226 ± 0.012243069144373108, μ: 0.00038165827270340245 ± 0.014994576804522203
Red Particle σ: 0.19514856444555856 ± 0.004919473860240537, μ: 0.0040682468147739065 ± 0.00602510062559018

 Windowed turning angles analysis


100%|██████████| 288/288 [00:50<00:00,  5.75it/s]


In [10]:
blue_fit_wind_smooth

array([[4.16523963, 0.01679473],
       [4.1228829 , 0.01599424],
       [4.0708972 , 0.01492161],
       [4.03561493, 0.01416229],
       [4.01705882, 0.01357084],
       [3.98626345, 0.01326023],
       [3.94364223, 0.01318744],
       [3.91417166, 0.01297935],
       [3.89811457, 0.01228537],
       [3.8814764 , 0.01191168],
       [3.85375139, 0.01202269],
       [3.82639435, 0.01195421],
       [3.79420246, 0.01145383],
       [3.79139291, 0.01099608],
       [3.76642887, 0.01108051],
       [3.74712582, 0.01079247],
       [3.73208523, 0.01050736],
       [3.71781296, 0.01066622],
       [3.70645768, 0.01069752],
       [3.69711568, 0.01049469],
       [3.69474605, 0.01069041],
       [3.68018961, 0.01029578],
       [3.66532848, 0.01020216],
       [3.65951902, 0.00983527],
       [3.65170022, 0.00985649],
       [3.64027697, 0.00952474],
       [3.63084516, 0.00945261],
       [3.64784322, 0.00931691],
       [3.64846509, 0.00925782],
       [3.63344332, 0.00907588],
       [3.

# VELOCITY AUTOCORRELATION

In [6]:
show_verb = False
run_windowed_analysis = False
overwrite = False
animated_plot_verb = False

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

Velocity Autocorrelation Analysis: show_verb = False, run_windowed_analysis = False, animated_plot_verb = False
Global Velocity Autocovariance Function
Windowed analysis


# RADIAL DISTRIBUTON FUNCTION

In [7]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

print("Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}
Radial Distribution Function 
Radial Distribution Function from the center


# 3 BODY STRUCTURE METRIC

In [8]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb

Calculating 3-body angles for raw trajs...


100%|██████████| 32000/32000 [00:15<00:00, 2038.38it/s]


Calculating 3-body angles for smooth trajs...


100%|██████████| 32000/32000 [00:15<00:00, 2129.33it/s]
